<a href="https://colab.research.google.com/github/LEE-I-HUA/antitrust/blob/CHRIS/Opn_Coun_Jud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# Opinion
import fitz
import re
import csv

pdf_file_path = '/content/drive/MyDrive/textmining/State Surperior Court (CA) (30).PDF'
csv_file_path = '/content/drive/MyDrive/textmining/opinion/opinion_Surperior.csv'

pattern = r'(Counsel:.*?|Judges:.*?(?=Opinion by:)|Opinion by:.*?)((?=Counsel:)|(?=Judges:)|(?=Opinion by:)|(?=Opinion)|$)'

def find_page_num(text, match_start, page_delimiters):
    for i, delimiter in enumerate(page_delimiters):
        if match_start < delimiter:
            return i
    return len(page_delimiters)

counsel_count = 0
judges_count = 0
opinion_by_count = 0

with fitz.open(pdf_file_path) as doc:
    all_text = ""
    page_delimiters = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        all_text += page.get_text("text") + "\f"
        page_delimiters.append(len(all_text))

    all_text = re.sub(r'Page \d+ of \d+', '', all_text)

    all_text = re.sub(r'\d+ Cal\..*?\*{1,4}\d+; (?:\d{4} )?Cal\..*?\*{1,4}\d+', '', all_text)

    page_text = f'頁碼 {page_num + 1}:\n{page_text}'

    with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['頁碼', '內容類型', '內容'])

        matches = re.finditer(pattern, all_text, re.DOTALL)
        for match in matches:
            match_content = match.group(1).strip()
            match_type = match.group(2).strip()
            match_page = find_page_num(all_text, match.start(), page_delimiters)

            if 'Counsel:' in match_content:
                csv_writer.writerow([match_page + 1, 'Counsel', match_content])
                counsel_count += 1
            elif 'Judges:' in match_content:
                csv_writer.writerow([match_page + 1, 'Judges', match_content])
                judges_count += 1
            elif 'Opinion by:' in match_content:
                csv_writer.writerow([match_page + 1, 'Opinion by', match_content])
                opinion_by_count += 1


print(f"完成：{csv_file_path}")
print(f"Counsel 數量: {counsel_count}")
print(f"Judges 數量: {judges_count}")
print(f"Opinion by 數量: {opinion_by_count}")

完成：/content/drive/MyDrive/textmining/opinion/opinion_Surperior.csv
Counsel 數量: 14
Judges 數量: 26
Opinion by 數量: 26


In [20]:
# Explain
# !pip install PyMuPDF
import fitz
import re
import csv

pdf_file_path = '/content/drive/MyDrive/textmining/State Surperior Court (CA) (30).PDF'
csv_file_path = '/content/drive/MyDrive/textmining/opinion/opinion_Surperior.csv'

pattern = r'(Counsel:.*?|Judges:.*?(?=Opinion by:)|Opinion by:.*?)((?=Counsel:)|(?=Judges:)|(?=Opinion by:)|(?=Opinion)|$)'

# " (Counsel:.*?|Judges:.*?(?=Opinion by:)|Opinion by:.*?) "  :
# capture content of " Counsel: ", " Judges: " 到 "Opinion by:" or "Opinion"

# " ((?=Counsel:)|(?=Judges:)|(?=Opinion by:)|(?=Opinion)|$) " :
# capture the type of the match ("Counsel:", "Judges:", "Opinion by:", "Opinion", or the end of the string).


def find_page_num(text, match_start, page_delimiters):

#找出匹配文本再PDF的頁碼. Text(整個PDF文本內容) . match_start(當前匹配文本起始位置)
# page_delimiters(包含每頁結尾在text中的列表) 將每頁文本加入all_text並記錄長度, 每個元素代表文本結束的位置
    for i, delimiter in enumerate(page_delimiters): # i為當前頁碼索引 從0開始
        if match_start < delimiter: #檢查match_start是否小於delimiter  , 小於代表匹配文本位於這一頁或之前的頁面
            return i #頁碼通常從1開始, 所以後續都會對這個索引 + 1
    return len(page_delimiters) # 表示匹配文本位於文檔最後一頁

# 加入計數器
counsel_count = 0
judges_count = 0
opinion_by_count = 0

with fitz.open(pdf_file_path) as doc:
    all_text = "" #儲存整個文本內容
    page_delimiters = [] #儲存每頁文本結束的列表

    for page_num in range(len(doc)): #閱讀文本每一頁 . len(doc):總頁數
        page = doc.load_page(page_num) #載入當前頁
        all_text += page.get_text("text") + "\f" #從當前頁提取文本並添加到all_text中 # 使用分頁符號 "\f" 作為分頁標記
        page_delimiters.append(len(all_text))  # 在page_delimiters列表添加累積文本長度,也就是當前頁結束位置, 紀錄每頁結束的位置.

    # 移除包含 "Page \d+ of \d+" 的字樣
    all_text = re.sub(r'Page \d+ of \d+', '', all_text)

# re.sub 查找和替換 符合正則表達式的字串, ''表示將匹配內容刪除
# \d+ 數字 . Page和of 為單詞 . 表 刪除 ex Page 23 of 456 將其中all_text中刪除

    # 移除其他不重要的字樣
    all_text = re.sub(r'\d+ Cal\..*?\*{1,4}\d+; (?:\d{4} )?Cal\..*?\*{1,4}\d+', '', all_text)

# Cal\. = Cal. " .*? " 非貪婪匹配任所有字符直到遇到後面的模式 "\*{1,4}" 匹配1~4個'*' ;匹配分號
# (?:\d{4} ) 匹配四位數字後跟一個空格 , Cal\..*?\*{1,4}\d+ 同上 匹配Cal. 後面的文字,星號,數字

    # 在每段文本前加上頁碼標記
    page_text = f'頁碼 {page_num + 1}:\n{page_text}'

    with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
      #確保文件使用utf-8編碼, newline=''為了避免在不同操作系統寫入時產生新行問題
        csv_writer = csv.writer(csv_file) # 創建CSV寫入器,將數據輸入
        csv_writer.writerow(['頁碼', '內容類型', '內容']) #將標題寫入

        matches = re.finditer(pattern, all_text, re.DOTALL) # find all occourance of pattern in 'text'
        for match in matches: #loop initiate matches from re.finditer
            match_content = match.group(1).strip() # 提取第一個匹配組內容,去除首尾空格
            match_type = match.group(2).strip()  # 提取第二個匹配組內容,去除首尾空格
            match_page = find_page_num(all_text, match.start(), page_delimiters)
            # 調用find_page_num函數確定匹配內容所在頁碼,將這些資訊寫入CSV

            if 'Counsel:' in match_content:
                csv_writer.writerow([match_page + 1, 'Counsel', match_content])
                counsel_count += 1
            elif 'Judges:' in match_content:
                csv_writer.writerow([match_page + 1, 'Judges', match_content])
                judges_count += 1
            elif 'Opinion by:' in match_content:
                csv_writer.writerow([match_page + 1, 'Opinion by', match_content])
                opinion_by_count += 1

print(f"完成：{csv_file_path}")
print(f"Counsel 數量: {counsel_count}")
print(f"Judges 數量: {judges_count}")
print(f"Opinion by 數量: {opinion_by_count}")

完成：/content/drive/MyDrive/textmining/opinion/opinion_Surperior.csv
Counsel 數量: 14
Judges 數量: 26
Opinion by 數量: 26


In [ ]:
from google.colab import drive
drive.mount('/content/drive')